# SOLUZIONE OTTIMA

## Import

In [5]:
import gurobipy as gp
from gurobipy import GRB

## Funzioni per la gestione dell'input e dell'output

In [6]:
def read_input(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    R, S, U, P, M = map(int, lines[0].strip().split())
    unavailable_slots = []

    for i in range(1, U + 1):
        unavailable_slots.append(tuple(map(int, lines[i].strip().split())))

    servers = []
    for i in range(U + 1, U + 1 + M):
        servers.append(tuple(map(int, lines[i].strip().split())))

    return R, S, U, P, M, unavailable_slots, servers

def write_output(allocation, file_path, score, score_path):
    with open(file_path, 'w') as f:
        for item in allocation:
            if item == 'x':
                f.write('x\n')
            else:
                f.write(f"{item[0]} {item[1]} {item[2]}\n")
    with open(score_path, 'w') as s:
        s.write(f"{score}\n")

## Lettura dei file di input

In [7]:
#path = "input_output/hashcode_2015_qualification_round.in"
path = "input_output/input.txt"
R, S, U, P, M, unavailable_slots, servers = read_input(path)

## Creazione del modello

In [8]:
model = gp.Model("hashcode_pli")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-15


### 1. Variabili di decisione

In [9]:
x = model.addVars(M, R, S, vtype=GRB.BINARY, name="x")
y = model.addVars(M, P, vtype=GRB.BINARY, name="y")
z = model.addVars(P, vtype=GRB.CONTINUOUS, name="z")

### 2. Vincoli

In [10]:
model.addConstrs(gp.quicksum(x[i, r, s] for r in range(R) for s in range(S)) == 1 for i in range(M))

model.addConstrs(gp.quicksum(x[i, r, s] for i in range(M)) <= 1 for r in range(R) for s in range(S))

model.addConstrs(gp.quicksum(y[i, p] for p in range(P)) == 1 for i in range(M))

for r, s in unavailable_slots:
    for i in range(M):
        model.addConstr(x[i, r, s] == 0, f"unavailable_{i}_{r}_{s}")

for i, (size, budget) in enumerate(servers):
    for r in range(R):
        for s in range(S):
            if (s + size > S):
                for offset in range(size):
                    if (s + offset < S):
                        model.addConstr(x[i, r, s + offset] == 0, f"space_{i}_{r}_{s}")

for p in range(P):
    c_tot = gp.quicksum(servers[i][1] * y[i, p] for i in range(M))
    
    for r in range(R):
        c_row = gp.quicksum(servers[i][1] * x[i, r, s] * y[i,p] for i in range(M) for s in range(S))
        model.addConstr(z[p] <= c_tot - c_row, name="z_constr")

### 3. Funzione obiettivo

In [11]:
model.setParam('OutputFlag', 1)

model.setObjective(gp.quicksum(z[p] for p in range(P)), GRB.MAXIMIZE)

## Calcolo della soluzione

In [12]:
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 30 rows, 50 columns and 100 nonzeros
Model fingerprint: 0xeae5cab7
Model has 4 quadratic constraints
Variable types: 2 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+01, 2e+01]
  QLMatrix range   [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 17 rows and 13 columns
Presolve time: 0.01s
Presolved: 110 rows, 68 columns, 392 nonzeros
Variable types: 0 continuous, 68 integer (66 binary)
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 20.0000000

Root relaxation: objective 2.750000e+01, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes

In [13]:
score = 1000000000
if model.status == GRB.OPTIMAL:
    for p in range(P):
        print(f"Capacità minima garantita per il pool {p}: {z[p].X}")
        score = z[p].X if z[p].X < score else score
score = int(score)
        
Xvals = model.getAttr('X', x)
Yvals = model.getAttr('X', y)

Capacità minima garantita per il pool 0: 10.0
Capacità minima garantita per il pool 1: 15.0


## Elaborazione dell'output

In [14]:
allocation = []

for i in range(M):
    assigned = False
        
    for r in range(R):
        for s in range(S):
            if Xvals[i,r,s] == 1:
                for p in range(P):
                    if Yvals[i,p] == 1:
                        allocation.append((r, s, p))
                        assigned = True
                        break
            if assigned:
                break
        if assigned:
            break
    if not assigned:
        allocation.append('x')

write_output(allocation, "input_output/outputOpt.txt", score, "input_output/score_opt.txt")
print(f"Score: {score}")

Score: 10
